## Zero-shot text classification using IBM Watson.ai


In [62]:
!pip install wget | tail -n 1

In [63]:
!pip install scikit-learn | tail -n 1
!pip install "ibm-watson-machine-learning>=1.0.310" | tail -n 1

In [91]:
url = "https://us-south.ml.cloud.ibm.com"
apiKey = "" # api key is to be entered here

In [66]:
credentials = {
    "url" : url,
    "apikey" : apiKey
}

In [92]:
import os

try:
    project_id = os.environ["PROJECT_ID"]
except KeyError:
    project_id = input("Enter your project id")

project_id

'f925dd0c-2679-4843-8881-9a03fcc103f5'

## IBM CloudObject to create an object in cloud to fetch dataset

In [68]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='', # ibm cloudObject Key to be intered here
    ibm_auth_endpoint="https://iam.cloud.ibm.com/identity/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.direct.us-south.cloud-object-storage.appdomain.cloud')

bucket = 'sentimentalanalysis-donotdelete-pr-xwbdkhjkianjn3'
object_key = 'data.csv'

body = cos_client.get_object(Bucket=bucket,Key=object_key)['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

df_1 = pd.read_csv(body)
df_1.head(10)


,Sentence,Sentiment
0,The GeoSolutions technology will leverage Bene...,positive
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",negative
2,"For the last quarter of 2010 , Componenta 's n...",positive
3,According to the Finnish-Russian Chamber of Co...,neutral
4,The Swedish buyout firm has sold its remaining...,neutral
5,$SPY wouldn't be surprised to see a green close,positive
6,Shell's $70 Billion BG Deal Meets Shareholder ...,negative
7,SSH COMMUNICATIONS SECURITY CORP STOCK EXCHANG...,negative
8,Kone 's net sales rose by some 14 % year-on-ye...,positive
9,The Stockmann department store will have a tot...,neutral


## Data Preprocessing

In [69]:
def mapper_function(value):
    if value == "positive":
        return 1;
    elif value == "negative":
        return -1;
    else:
        return 0;

In [70]:
df_1['Sentiment'] = df_1['Sentiment'].map(mapper_function)
df_1.head()

,Sentence,Sentiment
0,The GeoSolutions technology will leverage Bene...,1
1,"$ESI on lows, down $1.50 to $2.50 BK a real po...",-1
2,"For the last quarter of 2010 , Componenta 's n...",1
3,According to the Finnish-Russian Chamber of Co...,0
4,The Swedish buyout firm has sold its remaining...,0


In [71]:
label_map = {
    -1 : "negative",
    0: "neutral",
    1: "postive"
}

In [72]:
df_1.value_counts(['Sentiment'])

Sentiment
 0           3130
 1           1852
-1            860
Name: count, dtype: int64

In [73]:
from sklearn.model_selection import train_test_split

data_train, data_test, y_train, y_test = train_test_split(
    df_1['Sentence'],
    df_1['Sentiment'],
    test_size = 0.3,
    random_state = 33,
    stratify = df_1['Sentiment'])

data_train = pd.DataFrame(data_train)
data_test = pd.DataFrame(data_test)

data_train.head()

,Sentence
5496,The aim is to convert the plants into flexible...
1051,"( ADP News ) - Feb 4 , 2009 - Finnish broadban..."
4999,A total of 185 Wonderware Certified SIs are av...
778,The contracts comprise turnkey orders for RoRo...
2470,"A total of 1,800,000 stock options were issued..."


## Choosing models from IBM Watson.ai

- here we choose FLAN_T5_XXL

In [96]:
from ibm_watson_machine_learning.foundation_models.utils.enums import ModelTypes
print([model.name + "\n" for model in ModelTypes])

['FLAN_T5_XXL\n', 'FLAN_UL2\n', 'MT0_XXL\n', 'GPT_NEOX\n', 'MPT_7B_INSTRUCT2\n', 'STARCODER\n', 'LLAMA_2_70B_CHAT\n', 'LLAMA_2_13B_CHAT\n', 'GRANITE_13B_INSTRUCT\n', 'GRANITE_13B_CHAT\n', 'FLAN_T5_XL\n', 'GRANITE_13B_CHAT_V2\n', 'GRANITE_13B_INSTRUCT_V2\n', 'ELYZA_JAPANESE_LLAMA_2_7B_INSTRUCT\n', 'MIXTRAL_8X7B_INSTRUCT_V01_Q\n', 'CODELLAMA_34B_INSTRUCT_HF\n', 'GRANITE_20B_MULTILINGUAL\n']


In [75]:
model_id = ModelTypes.FLAN_T5_XXL

In [76]:
from ibm_watson_machine_learning.metanames import GenTextParamsMetaNames as GenParams

parameters = {
    GenParams.DECODING_METHOD: "greedy",
    GenParams.RANDOM_SEED: 33,
    GenParams.REPETITION_PENALTY:1,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.MAX_NEW_TOKENS: 1
}

In [77]:
from ibm_watson_machine_learning.foundation_models import Model

model = Model(
    model_id = model_id,
    credentials = credentials,
    params= parameters,
    project_id = project_id
)

/opt/conda/envs/Python-RT24.1/lib/python3.11/site-packages/ibm_watson_machine_learning/foundation_models/utils/utils.py:273: LifecycleWarning: Model 'google/flan-t5-xxl' is in deprecated state from 2025-05-28 until 2025-07-30. IDs of alternative models: None. Further details: https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-model-lifecycle.html?context=wx&audience=wdp
  warnings.warn(default_warning_template.format(


In [78]:
instruction = """
 determine the sentiment of the sentense.
 use either 'positive', 'negative', 'neutral'.
 use the provided example as an templeate.
 """

In [79]:
zero_shot_inputs = [{"input": text} for text in data_test['Sentence']]
for i in range(3):
    print(f"THe sentence example {i+1} is: \n\t {zero_shot_inputs[i]['input']}\n")

THe sentence example 1 is: 
	 `` I 'm pleased to receive the Nomination Committee 's request and confidence , '' says Jon Risfelt .

THe sentence example 2 is: 
	 : Lietuvos Respublikos sveikatos apsaugos ministerija has awarded contract to UAB `` AFFECTO LIETUVA '' for financial systems software package .

THe sentence example 3 is: 
	 The company said production volumes so far indicate the circuit is capable of the targeted output rate of 60,000 tonnes per day , or 22 million tonnes a year .



In [80]:
data_train_and_labels = data_train.copy()
data_train_and_labels['Sentiment'] = y_train

In [81]:
few_shot_example = []
few_shot_examples = []
for sentence, sentiment in data_train_and_labels.groupby('Sentiment')\
                                            .apply(lambda x:x.sample(3)).values:
    few_shot_example.append(f"\tsentence:\t{sentence}\n\tsenteiment:{sentiment}\n")
few_shot_examples = [ " ".join(few_shot_example)]

In [82]:
few_shot_inputs_ = [{"input": text} for text in data_test['Sentence'].values]

In [83]:
results = []
for inp in few_shot_inputs_[:3]:
    results.append(model.generate(" ".join([instruction + few_shot_examples[0], inp['input']]))["results"][0])

In [93]:
import json
print(json.dumps(results, indent=3))

[
   {
      "generated_text": "positive",
      "generated_token_count": 1,
      "input_token_count": 421,
      "stop_reason": "max_tokens"
   },
   {
      "generated_text": "neutral",
      "generated_token_count": 1,
      "input_token_count": 443,
      "stop_reason": "max_tokens"
   },
   {
      "generated_text": "positive",
      "generated_token_count": 1,
      "input_token_count": 423,
      "stop_reason": "max_tokens"
   }
]


## Model predictions


In [94]:
y_true = [ label_map[label] for label in y_test.values[:3]]   # actuall values
y_true

['postive', 'postive', 'neutral']

In [95]:
y_pred = [result['generated_text'] for result in results]  # predicted value
y_pred

['positive', 'neutral', 'positive']